In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk.corpus import stopwords

In [111]:
#load dataset and replace "Unknown" cells with nan""
anime = pd.read_csv(r"C:\Users\pvans\Downloads\archive (4)\anime.csv", na_values = ["Unknown"])
synopsis = pd.read_csv(r"C:\Users\pvans\Downloads\archive (4)\anime_with_synopsis.csv", na_values = ["Unknown"])


# Cleaning and Pre

In [112]:
anime = anime.merge(synopsis, on = ["MAL_ID", "Name", "Score", "Genres"])

In [113]:
anime.head(2)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26.0,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,"In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1.0,"Sep 1, 2001",NaN,...,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0,"other day, another bounty—such is the life of ..."


In [114]:
anime.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1', 'sypnopsis'],
      dtype='object')

In [115]:
anime.fillna("", inplace = True)

# Content

In [116]:
unit_features=['Name','Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1', 'sypnopsis']

In [117]:
tfd = TfidfVectorizer()
tfid_matrix = tfd.fit_transform(anime[unit_features])
print(tfid_matrix)

  (0, 13)	1.0
  (1, 21)	1.0
  (2, 8)	1.0
  (3, 5)	0.7725740835653588
  (3, 13)	0.6349246297027276
  (4, 10)	0.7725740835653588
  (4, 13)	0.6349246297027276
  (5, 26)	1.0
  (6, 6)	1.0
  (7, 1)	1.0
  (8, 17)	1.0
  (9, 18)	1.0
  (10, 11)	1.0
  (11, 23)	1.0
  (12, 22)	1.0
  (13, 4)	1.0
  (14, 20)	1.0
  (15, 19)	1.0
  (16, 16)	1.0
  (17, 12)	1.0
  (18, 7)	1.0
  (19, 28)	1.0
  (20, 2)	1.0
  (21, 9)	0.7071067811865476
  (21, 14)	0.7071067811865476
  (22, 3)	1.0
  (23, 27)	0.5773502691896257
  (23, 25)	0.5773502691896257
  (23, 15)	0.5773502691896257
  (24, 0)	0.880112305155453
  (24, 21)	0.47476555299848255
  (25, 21)	1.0
  (26, 21)	1.0
  (27, 21)	1.0
  (28, 21)	1.0
  (29, 21)	1.0
  (30, 21)	1.0
  (31, 21)	1.0
  (32, 21)	1.0
  (33, 21)	1.0
  (34, 24)	1.0


In [118]:
# get similarity anime matrix
cos_sim = cosine_similarity(tfid_matrix, tfid_matrix)
print(cos_sim)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 1. 1. 0.]
 [0. 1. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [119]:
title_ind = pd.DataFrame(anime.index, index = anime.Name)
title_ind.loc["Initial D Fourth Stage"]

0    8
Name: Initial D Fourth Stage, dtype: int64

In [120]:
# func for getting recommendation
def get_recomendation(title, n = 100, cos_sim = cos_sim):
    if title in title_ind.index:
        indx = title_ind.loc[title] 
        similar = list(enumerate(cos_sim[indx][0]))
        top_sim = sorted(similar, reverse = True, key = lambda x: x[1])
        return [anime.loc[x[0]]["Name"] for x in top_sim[1:n]]
    return "There is no such an anime"
        

In [121]:
print(get_recomendation("Akira", 9))

['Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen', 'Rurouni Kenshin: Meiji Kenkaku Romantan', 'Rurouni Kenshin: Meiji Kenkaku Romantan - Ishinshishi e no Chinkonka', 'Akira', '.hack//Sign', 'Aa! Megami-sama!', 'Aa! Megami-sama! (TV)', 'Tenshi Kinryouku']


# Collaborative

In [ ]:
#data of rating table is too huge cannot import so clled limited rows

In [138]:
file_path = r"C:\Users\pvans\Downloads\archive (4)\rating_complete.csv"

# Specify the number of rows you want to read
num_rows_to_read = 17563  # Change this to the desired number of rows

# Read the specified number of rows from the CSV
rating_df = pd.read_csv(file_path, nrows=num_rows_to_read)

In [139]:
rating_df = rating_df[['anime_id','rating']]
rating_df = rating_df.rename(columns={'anime_id':'MAL_ID'})
#anime = anime.merge(rating_df, on = ['MAL_ID'])


In [188]:
moviemat = rating_df.pivot_table(index='MAL_ID', columns='MAL_ID', values='rating')
moviemat = moviemat.replace('NaN', '0')

moviemat.head(10)

MAL_ID,1,5,6,7,8,15,16,17,18,19,...,41353,41380,41389,41468,41619,42260,42571,42893,42990,45753
MAL_ID,,,,,,,,,,,,,,,,,,,,,
1,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,8.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,8.545455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,8.666667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,8.75,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.166667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#used anime_name to get name from the anime tableb cannot merge tables as rows going beyound 3lakh so math mailid 
#and retrieve name

In [166]:
moviemat2 = moviemat.fillna(0)
film = 578
anime_name = anime.loc[anime["MAL_ID"] == film, "Name"].values[0]
rating = moviemat2[film]
film_corr = pd.DataFrame(moviemat2.corrwith(rating), 
                           columns=['Correlation']).sort_values('Correlation', ascending=False)
film_corr.head(10)

,Correlation
MAL_ID,
578,1.000000
2952,-0.000255
49,-0.000255
76,-0.000255
136,-0.000255
139,-0.000255
4059,-0.000255
232,-0.000255
242,-0.000255


In [184]:
def find_me_film( film):
    anime_name = anime.loc[anime["MAL_ID"] == film, "Name"].values[0]
    
    rating = moviemat2[film]
    film_corr = pd.DataFrame(moviemat2.corrwith(rating), 
                           columns=['Correlation']).sort_values('Correlation', ascending=False)
    print(f' {anime_name}:')
    
    
    for i, filmy in list(enumerate(film_corr.index[1:20])):
        anime_name = anime.loc[anime["MAL_ID"] == filmy, "Name"].values[0]
        
        print(f'{i+1}. {anime_name}')

In [185]:
find_me_film(121)

 Fullmetal Alchemist:
1. Full Moon wo Sagashite
2. InuYasha Movie 3: Tenka Hadou no Ken
3. Witchblade
4. Black Lagoon: The Second Barrage
5. Naruto: Shippuuden
6. Perfect Blue
7. Black Lagoon
8. Higurashi no Naku Koro ni
9. Ai Yori Aoshi
10. Paradise Kiss
11. K-On!: Live House!
12. Yuu☆Yuu☆Hakusho
13. Arakawa Under the Bridge
14. Hetalia World Series
15. Top wo Nerae! Gunbuster
16. Mahou Shoujo Madoka★Magica
17. Tsumiki no Ie
18. Hellsing
19. Seitokai Yakuindomo OVA


In [152]:
mal_id_to_find = 578
anime_name = anime.loc[anime["MAL_ID"] == mal_id_to_find, "Name"].values[0]
print(anime_name)

Hotaru no Haka
